#### Handling Imbalanced Dataset with Machine Learning

In [ ]:
import pandas as pd
df=pd.read_csv('ais_disabling_events_main_only_imp_col_csv.csv')
df.head()

,vessel_class,gap_hours,iuu_caught,spherical_distances,eez_check,Ocean List New whose false were in OG
0,other,13.016667,0,207.942845,0,atlantic
1,other,13.850000,0,9.704232,0,atlantic
2,other,33.733333,0,10.789069,1,atlantic
3,other,30.650000,0,11.695860,1,atlantic
4,other,50.283333,0,93.113335,1,atlantic


In [ ]:
df.shape

(55141, 6)

In [ ]:
df['iuu_caught'].value_counts()

0    55041
1      100
Name: iuu_caught, dtype: int64

In [ ]:
#### Independent and Dependent Features
X=df.drop("iuu_caught",axis=1)
y=df.iuu_caught

### One Hot Encoding

In [ ]:
print(df['vessel_class'].unique())
print(df['Ocean List New whose false were in OG'].unique())

['other' 'trawlers' 'squid_jigger' 'tuna_purse_seines'
 'drifting_longlines']
['atlantic' 'southern' 'mediterranean' 'arctic' 'pacific' 'indian']


In [ ]:
print(df['vessel_class'].value_counts())
print(df['Ocean List New whose false were in OG'].value_counts())

drifting_longlines    18608
squid_jigger          15965
tuna_purse_seines      8620
trawlers               7865
other                  4083
Name: vessel_class, dtype: int64
pacific          31870
atlantic         16469
indian            6176
arctic             499
southern           116
mediterranean       11
Name: Ocean List New whose false were in OG, dtype: int64


In [ ]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['vessel_class', 'Ocean List New whose false were in OG'])
print(one_hot_encoded_data.head())

   gap_hours  iuu_caught  spherical_distances  eez_check  \
0  13.016667           0           207.942845          0   
1  13.850000           0             9.704232          0   
2  33.733333           0            10.789069          1   
3  30.650000           0            11.695860          1   
4  50.283333           0            93.113335          1   

   vessel_class_drifting_longlines  vessel_class_other  \
0                                0                   1   
1                                0                   1   
2                                0                   1   
3                                0                   1   
4                                0                   1   

   vessel_class_squid_jigger  vessel_class_trawlers  \
0                          0                      0   
1                          0                      0   
2                          0                      0   
3                          0                      0   
4              

In [ ]:
#### Independent and Dependent Features
X=one_hot_encoded_data.drop("iuu_caught",axis=1)
y=one_hot_encoded_data.iuu_caught

#### SMOTETomek

In [ ]:
y.value_counts()

0    55041
1      100
Name: iuu_caught, dtype: int64

In [ ]:
# data wrangling
import pandas as pd
import numpy as np
# inputs data preparation
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
# modeling
from sklearn.linear_model import LogisticRegression
# model validation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
# number of fraud cases
frauds = len(df[df.iuu_caught == 1])
# selecting the indices of the non-fraud classes
fraud_indices = df[df.iuu_caught == 1].index
nonfraud_indices = df[df.iuu_caught == 0].index
# from all non-fraud observations, randomly select observations equal to number of fraud observations
random_nonfraud_indices = np.random.choice(nonfraud_indices, frauds, replace = False)
random_nonfraud_indices = np.array(random_nonfraud_indices)
# appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_nonfraud_indices])
# undersample dataset
under_sample_data = one_hot_encoded_data.iloc[under_sample_indices,:]
# now split X, y variables from the under sample data
X_undersample = under_sample_data.loc[:, under_sample_data.columns != 'iuu_caught']
y_undersample = under_sample_data.loc[:, under_sample_data.columns == 'iuu_caught']
print(X_undersample.value_counts())

gap_hours    spherical_distances  eez_check  vessel_class_drifting_longlines  vessel_class_other  vessel_class_squid_jigger  vessel_class_trawlers  vessel_class_tuna_purse_seines  Ocean List New whose false were in OG_arctic  Ocean List New whose false were in OG_atlantic  Ocean List New whose false were in OG_indian  Ocean List New whose false were in OG_mediterranean  Ocean List New whose false were in OG_pacific  Ocean List New whose false were in OG_southern
12.083333    67.277665            0          0                                1                   0                          0                      0                               0                                             1                                               0                                             0                                                    0                                              0                                                 1
123.066667   29.078697            0          0               

In [ ]:
# split dataset
X_train_undersample, X_test_undersample, y_train_undersample, y_test_undersample = train_test_split(X_undersample, y_undersample, test_size = 0.3, random_state = 0)

In [ ]:
print(X_train_undersample.value_counts())
print(y_train_undersample.value_counts())
print(X_test_undersample.value_counts())
print(y_test_undersample.value_counts())

gap_hours    spherical_distances  eez_check  vessel_class_drifting_longlines  vessel_class_other  vessel_class_squid_jigger  vessel_class_trawlers  vessel_class_tuna_purse_seines  Ocean List New whose false were in OG_arctic  Ocean List New whose false were in OG_atlantic  Ocean List New whose false were in OG_indian  Ocean List New whose false were in OG_mediterranean  Ocean List New whose false were in OG_pacific  Ocean List New whose false were in OG_southern
12.083333    67.277665            0          0                                1                   0                          0                      0                               0                                             1                                               0                                             0                                                    0                                              0                                                 1
117.850000   78.834114            1          0               

In [ ]:
# instantiate model
model = LogisticRegression()
# fit 
model.fit(X_train_undersample, y_train_undersample)
# predict
y_pred = model.predict(X_test_undersample)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
classification_report = classification_report(y_test_undersample, y_pred)
confusion_matrix = confusion_matrix(y_test_undersample, y_pred)
print("CLASSIFICATION REPORT")
print(classification_report)
print("CONFUSION MATRIX") 
print(confusion_matrix)

CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.87      0.65      0.74        31
           1       0.70      0.90      0.79        29

    accuracy                           0.77        60
   macro avg       0.79      0.77      0.76        60
weighted avg       0.79      0.77      0.76        60

CONFUSION MATRIX
[[20 11]
 [ 3 26]]
